In [1]:
import pandas as pd
# import tensorflow as tf
import numpy as np
import pickle
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
!pip install git+https://1c22a46e3b7202336e7cdd86aad07e1712df5b4d@github.com/ScudraServicos/scudra-libraries.git@master

In [ ]:
print(tf.__version__)

In [2]:
import unboxapi
from unboxapi.tasks import TaskType
from unboxapi.models import ModelType

In [3]:
client = unboxapi.UnboxClient("867d5a09-b923-4d99-8287-1ad97bb42105")

# Load data

In [4]:
data = pd.read_csv("sample_behavior_data.csv")

In [5]:
data.head()

,days_of_account,purchases_count_24m,purchases_count_12m,purchases_count_6m,payments_count_24m,payments_count_12m,payments_count_6m,installments_count_24m,installments_count_12m,installments_count_6m,...,months_since_last_payment_24m,months_since_last_payment_12m,months_since_last_payment_6m,delayed_days_installments_max_24m,delayed_days_installments_max_12m,delayed_days_installments_max_6m,delayed_days_open_installments_max_24m,delayed_days_open_installments_max_12m,delayed_days_open_installments_max_6m,over60_6
0,5594.0,2.0,1.0,1.0,19.0,19.0,14.0,27.0,12.0,12.0,...,0.0,0.0,0.0,86.0,86.0,86.0,0.0,0.0,0.0,1
1,660.0,3.0,1.0,1.0,23.0,12.0,7.0,24.0,3.0,3.0,...,2.0,2.0,2.0,56.0,56.0,56.0,56.0,56.0,56.0,1
2,4728.0,2.0,0.0,0.0,17.0,12.0,4.0,17.0,0.0,0.0,...,1.0,1.0,1.0,13.0,13.0,13.0,0.0,0.0,0.0,1
3,5683.0,3.0,1.0,1.0,30.0,16.0,5.0,33.0,4.0,4.0,...,0.0,0.0,0.0,5.0,5.0,-2.0,0.0,0.0,0.0,1
4,145.0,1.0,1.0,1.0,4.0,4.0,4.0,14.0,14.0,14.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1


# Predict Scores

In [6]:
def clip_distribution(X, max_value=None, min_value=None):
    """
    Caps values on `X` to boundaries `max_value` and `min_value`.
    """
    if max_value is None:
        max_value = X.max()
    if min_value is None:
        min_value = X.min()

    clipped_X = []

    for x in X:
        if x > max_value:
            x = max_value
        elif x < min_value:
            x = min_value

        clipped_X.append(x)
    # Rishab: There was a return missing
    return clipped_X
    

def normalize_score_distribution(X, max_value=None, min_value=None):
    if max_value is None:
        max_value = X.max()
    if min_value is None:
        min_value = X.min()

    return (X - min_value) / (max_value - min_value)

    
def normalize_and_clip_predictions(predictions, max_value, min_value):
    predictions = np.array(predictions).flatten()
    clipped_preds = np.array(
        clip_distribution(predictions, max_value, min_value))
    normalized_preds = normalize_score_distribution(clipped_preds, max_value,
                                                    min_value)
    return np.array([[pred] for pred in normalized_preds])

In [ ]:
# Load model
ensemble_model = tf.keras.models.load_model("model.h5")

# Load data_processor
with open("data_processor.pkl", "rb") as fin:
    dataprocessor = pickle.load(fin)
    
# Load metadata
with open("training_metadata.pkl", "rb") as fin:
    metadata = pickle.load(fin)
    
dataprocessor.get_metadata()

In [ ]:
def predict_proba(model, input_features: np.ndarray, dataprocessor, normalize_and_clip_predictions):
    # Rishab: The line below isn't needed since input features will be a numpy array with only the features
#     X_test = input_features[input_features.columns.values.tolist()[:-1]]
    # Trasform data
    X_test = dataprocessor.transform(input_features).reset_index(drop=True)

    probs_test = model.predict(X_test)
    probs_test = normalize_and_clip_predictions(
        probs_test, max_value=metadata.score_max_value, min_value=metadata.score_min_value
    )
    
    top = probs_test.reshape(-1)
    return [[1-x,x] for x in top]

In [7]:
sample = data[data.columns.values.tolist()[:-1]].sample(2).to_numpy()
sample

array([[1.37300000e+03, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.20000000e+01, 9.00000000e+00, 6.00000000e+00, 1.20000000e+01,
        0.00000000e+00, 0.00000000e+00, 1.81902000e+03, 0.00000000e+00,
        0.00000000e+00, 1.51585000e+02, 0.00000000e+00, 0.00000000e+00,
        1.51585000e+02, 1.51863333e+02, 1.52420000e+02, 4.00000000e+00,
        4.00000000e+00, 4.00000000e+00, 8.00000000e+00, 5.00000000e+00,
        2.00000000e+00, 9.50000000e+00, 9.50000000e+00, 9.50000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.20000000e+01,
        1.20000000e+01, 6.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 2.40000000e+01, 2.40000000e+01, 2.40000000e+01,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.21000000e+02, 2.00000000e+00, 2.00000000e+00, 1.00000000e+00,
       

In [ ]:
# Rishab: It's important to first test the predict function
predict_proba(ensemble_model, sample, dataprocessor, normalize_and_clip_predictions)

In [ ]:
model = client.add_model(
    function=predict_proba, 
    model=ensemble_model,
    model_type=ModelType.keras,
    task_type=TaskType.TabularClassification,
    class_names=["delinquent", "good"],
    name='[model] A behavior model',
    description='this is test model',
    feature_names=data.columns.values.tolist()[:-1],
    train_sample_df=data.sample(5000),
    train_sample_label_column_name='over60_6',
    categorical_features_map={},
    dataprocessor = dataprocessor,
    normalize_and_clip_predictions = normalize_and_clip_predictions,
    requirements_txt_file="requirements.txt",
    setup_script="setup.sh" # Rishab: Added a setup script with the scudra install, which was needed
)

model.to_dict()

In [8]:
dataset = client.add_dataframe(
    df=data.sample(5000),
    class_names=["delinquent", "good"],
    label_column_name='over60_6',
    name="[dataset] A behavior cohort",
    description='this is a test dataset',
    task_type=TaskType.TabularClassification,
    feature_names=data.columns.values.tolist()[:-1],
    categorical_features_map={},
)

dataset.to_dict()

{'categoricalFeaturesMap': {},
 'classNameCounts': None,
 'classNames': ['delinquent', 'good'],
 'columnNames': None,
 'columnSettings': None,
 'dateCreated': '2022-05-15T23:02:54.529422Z',
 'description': 'this is a test dataset',
 'featureNames': ['days_of_account',
  'purchases_count_24m',
  'purchases_count_12m',
  'purchases_count_6m',
  'payments_count_24m',
  'payments_count_12m',
  'payments_count_6m',
  'installments_count_24m',
  'installments_count_12m',
  'installments_count_6m',
  'financed_value_sum_24m',
  'financed_value_sum_12m',
  'financed_value_sum_6m',
  'financed_value_mean_24m',
  'financed_value_mean_12m',
  'financed_value_mean_6m',
  'payments_mean_24m',
  'payments_mean_12m',
  'payments_mean_6m',
  'payments_delayed_count_24m',
  'payments_delayed_count_12m',
  'payments_delayed_count_6m',
  'payments_on_day_count_24m',
  'payments_on_day_count_12m',
  'payments_on_day_count_6m',
  'payments_delayed_days_mean_24m',
  'payments_delayed_days_mean_12m',
  'paym